In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls
import os
os.chdir("drive/My Drive/Diplomski rad/")
!ls

drive  sample_data
data  docs  pooling  slow-fast	tb.ipynb


In [3]:
!ls

data  docs  pooling  slow-fast	tb.ipynb


In [0]:
os.chdir("data/")

In [0]:
frames=10

In [6]:
import tensorflow
from new_generator import FrameGenerator
import glob
import os
import pickle

os.chdir("./Videos/")
classes = [i.split(os.path.sep)[1] for i in glob.glob('train/*')]
classes.sort()
print(len(classes))
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = frames
BS = 4
EPOCHS = 20


glob_pattern='train/{classname}/*.avi'

data_aug = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.2,
    horizontal_flip=True,
    rotation_range=6,
    width_shift_range=.15,
    height_shift_range=.15)

t_list=None
v_list=None

with open("trainlist2","rb") as fp:
    t_list=pickle.load(fp)

with open("validlist2","rb") as fp:
    v_list=pickle.load(fp)

train = FrameGenerator(
    BS,
    NBFRAME,
    classes, 
    SIZE,
    None,
    CHANNELS,
    glob_pattern,
    data_aug,
    typ="train",
    prepared_data=t_list
)

valid = FrameGenerator(
    BS,
    NBFRAME,
    classes, 
    SIZE,
    None,
    CHANNELS,
    glob_pattern,
    None,
    typ="valid",
    prepared_data=v_list
)

Using TensorFlow backend.


101
Loaded 7185 files for train generator.
Loaded 2352 files for valid generator.


In [0]:
import tensorflow
#from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam

frames = NBFRAME
base_model = MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)
#for layer in base_model.layers:
#  layer.trainable=False
model = Sequential()
model.add(TimeDistributed(base_model,input_shape=(frames,224,224,3)))
model.add(TimeDistributed(Flatten()))
model.add(GlobalMaxPooling1D())
model.add(Dense(101, kernel_regularizer=tensorflow.keras.regularizers.l2(0.1)))
model.add(Dense(101, kernel_regularizer=tensorflow.keras.regularizers.l2(0.1), activation='softmax'))
model.summary()

checkpoint = ModelCheckpoint("../../pooling/mobilenetv2-10frames/", monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')
opt = Adam(0.0001)#0.0001
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
print(len(valid)//valid.batch_size)
history = model.fit(
    train,
    steps_per_epoch=len(train),
    validation_data=valid,
    validation_steps=len(valid),
    verbose=1,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)
#model.fit_generator(train,validation_data=valid, epochs=EPOCHS, callbacks=[checkpoint])

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 7, 7, 1280)    2257984   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 62720)         0         
_________________________________________________________________
global_max_pooling1d (Global (None, 62720)             0         
_________________________________________________________________
dense (Dense)                (None, 101)               6334821   
_________________________________________________________________
dense_1 (Dense)              (None, 101)               10302     
Total params: 8,603,107
Trainable params: 8,568,995
Non-trainable params: 34,112
_________________________________________________________________
147
Epoch 1/20
1796/1796 [==================

Resnet 18 test

In [7]:
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import load_model

model = load_model("../../pooling/rs18-finaltry/")

checkpoint = ModelCheckpoint("../../pooling/rs18-finalfurt/", monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')
#opt = Adam(0.0001)#0.0001
#model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
print(len(valid)//valid.batch_size)
history = model.fit(
    train,
    steps_per_epoch=len(train),
    validation_data=valid,
    validation_steps=len(valid),
    verbose=1,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

147
Epoch 1/20
1796/1796 [==============================] - ETA: 0s - loss: 0.9390 - accuracy: 0.9056Invalid frame count for video train/PushUps/v_PushUps_g16_c04.avi . Requested: 32 Obtained: 29
No other option for train/PushUps/v_PushUps_g16_c04.avi
Could not get frames for train/PushUps/v_PushUps_g16_c04.avi

Epoch 00001: val_loss improved from inf to 1.07733, saving model to ../../pooling/rs18-finalfurt/
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../pooling/rs18-finalfurt/assets
1796/1796 [==============================] - 10221s 6s/step - loss: 0.9390 - accuracy: 0.9056 - val_loss: 1.0773 - val_accuracy: 0.8958
Epoch 2/20
1796/1796 [==============================] - ETA: 0s - loss: 0.8086 - accuracy: 0.9271Invalid frame count for video train/PushUps/v_PushUps_g16_c04.avi . Requested: 32 Obtained: 29
No other option for train/PushUps/v_PushUps_g16_c04.avi
Could not get frames for train/PushUps/v_PushUps_g16

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [8]:
!pip install git+https://github.com/qubvel/classification_models.git

from classification_models.tfkeras import Classifiers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam
 
frames = NBFRAME
ResNet18, preprocess_input = Classifiers.get('resnet18')
base_model = ResNet18((224, 224, 3), weights='imagenet', include_top=False)

#for layer in base_model.layers:
#  layer.trainable=False
model = Sequential()
model.add(TimeDistributed(base_model,input_shape=(frames,224,224,3)))
model.add(TimeDistributed(Flatten()))
model.add(GlobalMaxPooling1D())
model.add(Dense(101, kernel_regularizer=tensorflow.keras.regularizers.l2(0.01)))
model.add(Dense(101, kernel_regularizer=tensorflow.keras.regularizers.l2(0.01), activation='softmax'))
model.summary()

checkpoint = ModelCheckpoint("../../pooling/rs18-finaltry/", monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')
opt = Adam(0.0001)#0.0001
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
print(len(valid)//valid.batch_size)
history = model.fit(
    train,
    steps_per_epoch=len(train)
    validation_data=valid,
    validation_steps=len(valid),
    verbose=1,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-jup014r3
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-jup014r3
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20032 sha256=25715440a76e97914884ab8a8bcd6654a22ebbbc849958c12a7be26c5b5b48f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ilsfadhg/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 32, 7, 7, 512)     11186889  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 25088)         0         
_________________________________________________

KeyboardInterrupt: ignored